## Feature Engineering EDA ##

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import pearsonr

# Load the dataset
df = pd.read_csv("consolidated_dataset.csv")
df["ZIP Code"] = df["ZIP Code"].astype(str).str.zfill(5)

# Define independent variables
independent_vars = {
    "Losses/Premium": df["ratio_losses_to_premium"],
    "Renewed / Nonrenewed": df["ratio_new_to_nonrenewed"],
    "(New + Renewed) / Nonrenewed": (df["new_policies"] + df["renewed_policies"]) / df["nonrenewed_policies"],
    "(New + Renewed) / Total Units": (df["new_policies"] + df["renewed_policies"]) / df["total_units"],
    "log(Avg Median Income 2018–2021)": np.log(df["Avg_Median_Income_2018_2021"]),
    "Housing Value / Median Income": df["growth_exposure"] / df["Avg_Median_Income_2018_2021"],
    "Housing Value / log(Median Income)": df["growth_exposure"] / np.log(df["Avg_Median_Income_2018_2021"]),
    "Fire Smoke Claims / Total Units": df["fire_smoke_claims"] / df["total_units"],
    "(New + Renewed)/Nonrenewed * Income Growth": ((df["new_policies"] + df["renewed_policies"]) / df["nonrenewed_policies"]) * df["Total_Growth_2018_2021"],
    "(New + Renewed)/Nonrenewed / Income Growth": ((df["new_policies"] + df["renewed_policies"]) / df["nonrenewed_policies"]) / df["Total_Growth_2018_2021"],
    "Losses / Company Nonrenewed": df["fire_smoke_losses"] / df["company_nonrenewed"],
    "Losses / Owner Nonrenewed": df["fire_smoke_losses"] / df["owner_nonrenewed"],
    "Change Renewed / Income Growth": df["change_renewed_policies"] / df["Total_Growth_2018_2021"],
    "Change Nonrenewed / Income Growth": (df["change_nonrenewed_owner"] + df["change_nonrenewed_company"]) / df["Total_Growth_2018_2021"],
    "Growth Losses / Sum 1yr": df["growth_fire_smoke_losses"] / df["Sum_1yr"],
    "Growth Losses / Sum 3yr": df["growth_fire_smoke_losses"] / df["Sum_3yr"],
    "Growth Losses / Sum 5yr": df["growth_fire_smoke_losses"] / df["Sum_5yr"],
    "Growth Losses / Sum 10yr": df["growth_fire_smoke_losses"] / df["Sum_10yr"],
    "Growth Losses / Avg 3yr": df["growth_fire_smoke_losses"] / df["Avg_3yr"],
    "Growth Losses / Avg 5yr": df["growth_fire_smoke_losses"] / df["Avg_5yr"],
    "Growth Losses / Avg 10yr": df["growth_fire_smoke_losses"] / df["Avg_10yr"],
    "Premium / Exposure": df["earned_premium"] / df["earned_exposure"],
    "Nonrenewal Rate": df["nonrenewed_policies"] / df["expiring_policies"],
    "Loss Burden per Unit": df["fire_smoke_losses"] / df["total_units"],
    "Loss Burden per Exposure": df["fire_smoke_losses"] / df["earned_exposure"],
    "Premium Adequacy": df["earned_premium"] / df["fire_smoke_losses"],
    "Renewal Resilience": df["renewed_policies"] / (df["renewed_policies"] + df["nonrenewed_policies"]),
    "Sum 1yr": df["Sum_1yr"],
    "Sum 3yr": df["Sum_3yr"],
    "Sum 5yr": df["Sum_5yr"],
    "Sum 10yr": df["Sum_10yr"],
    "Avg 3yr": df["Avg_3yr"],
    "Avg 5yr": df["Avg_5yr"],
    "Avg 10yr": df["Avg_10yr"],
    "White Population": df["Avg_Percentage_2018_2021"],
    "Median Income": df["Avg_Median_Income_2018_2021"],
    "Income Growth": df["Total_Growth_2018_2021"],
    "White Population Growth": df["Avg_Change_2018_2021"]
}

# Dependent variable
y = df["fair_plan_units"]

# Store results
results = []

# Analyze and plot
for ind_label, x in independent_vars.items():
    valid = x.notna() & y.notna() & np.isfinite(x) & np.isfinite(y)
    x_valid = x[valid]
    y_valid = y[valid]
    if len(x_valid) > 1:
        slope = np.polyfit(x_valid, y_valid, 1)[0]
        r, p = pearsonr(x_valid, y_valid)
        results.append((ind_label, slope, r, p))

# Save results to CSV
results_df = pd.DataFrame(results, columns=["Independent Variable", "Slope", "Pearson Correlation", "P-value"])
results_df.to_csv("correlation_results.csv", index=False)

# Plot top 5 lowest p-values
results_df_sorted = results_df.sort_values("P-value").head(5)
for _, row in results_df_sorted.iterrows():
    x = independent_vars[row["Independent Variable"]]
    valid = x.notna() & y.notna() & np.isfinite(x) & np.isfinite(y)
    x_valid = x[valid]
    y_valid = y[valid]
    plt.figure(figsize=(10, 6))
    plt.scatter(x_valid, y_valid, alpha=0.5)
    coef = np.polyfit(x_valid, y_valid, 1)
    poly_fn = np.poly1d(coef)
    plt.plot(np.sort(x_valid), poly_fn(np.sort(x_valid)), color="red", linestyle="--")
    plt.title(f"{row['Independent Variable']} vs Fair Plan Units\nSlope = {row['Slope']:.2f}, r = {row['Pearson Correlation']:.2f}, p = {row['P-value']:.2e}")
    plt.xlabel(row["Independent Variable"])
    plt.ylabel("Fair Plan Units")
    plt.grid(True)
    plt.tight_layout()
    filename = f"plot_{row['Independent Variable'].replace(' ', '_').replace('/', '_')}_vs_Fair_Plan_Units.png"
    plt.savefig(filename, dpi=300)
    plt.show()
    plt.close()

print("✅ Analysis complete. Results saved to 'correlation_results.csv' and top 5 plots saved.")
